In [1]:
#import library
import requests
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import base64

In [2]:
#Load data
data = pd.read_csv('data\heart_dataset.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
data_test = data.iloc[0:1, :-1].copy() # mengambil data
data_test

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1


In [4]:
# fungsi untuk mengubah baris DataFrame menjadi serialized
def row_to_serialized_example(row):
    feature = {
        'age': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['age'])])),
        'sex': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['sex'])])),
        'cp': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['cp'])])),
        'trestbps': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['trestbps'])])),
        'chol': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['chol'])])),
        'fbs': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['fbs'])])),
        'restecg': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['restecg'])])),
        'thalachh': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['thalachh'])])),
        'exang': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['exang'])])),
        'oldpeak': tf.train.Feature(float_list=tf.train.FloatList(value=[float(row['oldpeak'])])),  # oldpeak tetap float
        'slope': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['slope'])])),
        'ca': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['ca'])])),
        'thal': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(row['thal'])]))
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()


In [5]:
serialized_example = row_to_serialized_example(data_test.iloc[0])
base64_example = base64.b64encode(serialized_example).decode('utf-8')

# Membuat JSON untuk dikirim ke endpoint
json_data = json.dumps(
    {
        "signature_name": "serving_default", 
        "instances": [
            {"examples": {"b64": base64_example}}
        ]
    }
)

# Endpoint model untuk prediksi
endpoint = "http://localhost:8080/v1/models/heart_disease_prediction:predict" 

response = requests.post(endpoint, data=json_data)

predictions = response.json()["predictions"]
prediction = predictions[0][0]
threshold = 0.5  # ambang batas

# Menentukan hasil prediksi
if prediction >= threshold:
    result = "besar kemungkinan resiko serangan jantung"
else:
    result = "kecil kemungkinan resiko serangan jantung"

print(f"Hasil prediksi: {result}")

Hasil prediksi: besar kemungkinan resiko serangan jantung


In [6]:
# Mengecek hasil prediksi apabila ada error
if response.status_code == 200:
    predictions = response.json()["predictions"]
else:
    print(f"Error: {response.status_code}")
    print(response.json())
